# Tringulate points from knwon camera poses with Pycolmap


In [1]:
%reload_ext autoreload
%autoreload 2

from pathlib import Path

import pycolmap
from deep_image_matching.triangulation import (
    create_db_from_model,
    import_keypoints,
    import_matches,
    import_verifed_matches,
)
from deep_image_matching.utils import (
    OutputCapture,
)

# Define project directory
# root_path = Path("datasets/belv_20230725")
root_path = Path("datasets/belv_winter")

# Path to the images
image_dir = root_path / "images"

# Path to the pre-computed COLMAP reconstuction with the knwon poses
sfm_path = root_path / "results_superpoint+lightglue_bruteforce_quality_high"
sfm_db_path = sfm_path / "database.db"
sfm_rec_path = sfm_path / "reconstruction"

# Path to the dense matching results to be triangulated
dense_path = root_path / "results_roma_bruteforce_quality_medium"
features_h5 = dense_path / "features.h5"
matches_h5 = dense_path / "matches.h5"
pair_file = dense_path / "pairs.txt"

# Path to the output for the dense matching
dense_db_path = dense_path / "database_dense.db"
model_path = dense_path / "dense_model"
model_path.mkdir(exist_ok=True, parents=True)

# Do geometric verification of the dense features (with known camera poses)
do_geometric_verification = True
max_error = 4

In [2]:
# Import the sparse reconstruction
sfm_rec = pycolmap.Reconstruction(sfm_rec_path)

# Create an empty database from the sparse reconstruction
image_ids = create_db_from_model(sfm_rec, dense_db_path)

# Add keypoints to the database
import_keypoints(features_h5, image_ids, dense_db_path)

# Add matches to the database, but do not add two-view geometry
import_matches(
    matches_h5,
    image_ids,
    dense_db_path,
    pair_file,
    add_two_view_geometry=not do_geometric_verification,
)


if do_geometric_verification:
    # Run the geometric verification with the knwon camera poses and add the inliers matches to the database in the two-view geometry table
    import_verifed_matches(
        image_ids,
        sfm_rec,
        dense_db_path,
        features_h5,
        matches_h5,
        pair_file,
        max_error=max_error,
    )

Importing matches: 100%|██████████| 10/10 [00:00<00:00, 1322.67it/s]

2024-04-10 10:10:31 | [INFO    ] Performing geometric verification of the matches...



Importing verified matches: 100%|██████████| 4/4 [00:02<00:00,  1.41it/s]

2024-04-10 10:10:34 | [INFO    ] mean/med/min/max valid matches 50.29/61.79/3.48/87.35%.


In [3]:
# Run the triangulation with the known camera poses

# Define the options for the triangulation according to the IncrementalPipelineOptions available in pycolmap
# print(pycolmap.IncrementalPipelineOptions().summary())
opt = dict(
    triangulation=dict(
        ignore_two_view_tracks=False,
        min_angle=0.5,
    ),
)
verbose = True

with OutputCapture(verbose):
    with pycolmap.ostream():
        reconstruction = pycolmap.triangulate_points(
            sfm_rec,
            dense_db_path,
            image_dir,
            model_path,
            options=opt,
        )

I20240410 10:10:37.078500 720999 misc.cc:198] 
Loading database
I20240410 10:10:37.083670 720999 database_cache.cc:54] Loading cameras...
I20240410 10:10:37.083731 720999 database_cache.cc:64]  3 in 0.000s
I20240410 10:10:37.083747 720999 database_cache.cc:72] Loading matches...
I20240410 10:10:37.085050 720999 database_cache.cc:78]  10 in 0.001s
I20240410 10:10:37.085073 720999 database_cache.cc:94] Loading images...
I20240410 10:10:37.129077 720999 database_cache.cc:143]  5 in 0.044s (connected 5)
I20240410 10:10:37.129146 720999 database_cache.cc:154] Building correspondence graph...
I20240410 10:10:37.163204 720999 database_cache.cc:190]  in 0.034s (ignored 0)
I20240410 10:10:37.164541 720999 timer.cc:91] Elapsed time: 0.001 [minutes]
I20240410 10:10:37.184716 720999 misc.cc:198] 
Triangulating image #1 (0)
I20240410 10:10:37.184751 720999 sfm.cc:473] => Image sees 0 / 38735 points
I20240410 10:10:37.289250 720999 sfm.cc:478] => Triangulated 38735 points
I20240410 10:10:37.289290 7

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  9.772162e+04    0.00e+00    1.70e+03   0.00e+00   0.00e+00  1.00e+04        0    1.16e-01    4.70e-01
   1  8.318726e+04    1.45e+04    3.53e-01   4.51e+00   1.00e+00  3.00e+04        0    1.44e-01    6.15e-01


I20240410 10:10:38.528729 720999 misc.cc:205] 
Bundle adjustment report
------------------------
I20240410 10:10:38.528859 720999 bundle_adjustment.cc:942] 
    Residuals : 665644
   Parameters : 499233
   Iterations : 2
         Time : 0.651156 [s]
 Initial cost : 0.383155 [px]
   Final cost : 0.353515 [px]
  Termination : Convergence

I20240410 10:10:38.546653 720999 incremental_mapper.cc:175] => Completed observations: 0
I20240410 10:10:38.558491 720999 incremental_mapper.cc:178] => Merged observations: 0
I20240410 10:10:38.590533 720999 incremental_mapper.cc:160] => Filtered observations: 5
I20240410 10:10:38.590562 720999 sfm.cc:521] => Changed observations: 0.000015
I20240410 10:10:38.738544 720999 misc.cc:198] 
Extracting colors


In [4]:
# Export the model in ply and bundler format
reconstruction.export_PLY(model_path / "dense_model.ply")
reconstruction.export_bundler(model_path / "bundler.out", model_path / "bunlder_list.txt", skip_distortion=True)